# Imports

In [43]:
import pandas as pd
from datasets import Dataset

# Data Pre-processing

In [44]:
df = pd.read_csv(r"zu.gold.seg.data", delimiter='\t')
df.head(3)

,TOKEN,MORPH ANALYSIS,UPOS
0,Ukwengeza,u-kw-engez-a,V
1,kulokhu,ku-lokhu,CDEM
2,",",",",PUNC


In [45]:
# drop/remove the MORPH ANALYSIS column
df = df.drop(columns=['MORPH ANALYSIS'])
df.head(3)

,TOKEN,UPOS
0,Ukwengeza,V
1,kulokhu,CDEM
2,",",PUNC


In [46]:
# convert panda dataframe to HuggingFace dataset
dataset = Dataset.from_pandas(df)
print(dataset.features)

{'TOKEN': Value(dtype='string', id=None), 'UPOS': Value(dtype='string', id=None)}


In [59]:
data = dataset.train_test_split(test_size=0.2)

print(f"There are {len(data['train'])} tokens in the training set.")
print(f"There are {len(data['test'])} tokens in the testing set.\n")

print(f"First 3 training tokens:\n{data['train'][:3]}\n")
print(f"First 3 test tokens:\n{data['test'][:3]}")

There are 39277 tokens in the training set.
There are 9820 tokens in the testing set.

First 3 training tokens:
{'TOKEN': ['isebenze', 'ezinhle', 'ukuhambisa'], 'UPOS': ['V', 'ADJ', 'V']}

First 3 test tokens:
{'TOKEN': ['Izinsizakusebenza', 'lokuthi', 'balungisele'], 'UPOS': ['N', 'POSS', 'POSS']}
